## q-QLMS

In [ ]:
# If gspx is not installed, we add it to the path
import os, sys
gdir = os.path.dirname(os.getcwd())  # parent folder
sys.path.insert(0, gdir)

In [ ]:
import matplotlib.pyplot as plt
from gspx.utils.graph import make_sensor
from gspx.utils.display import plot_graph
from gspx.signals import QuaternionSignal
import numpy as np

from gspx.qgsp import QMatrix

In [ ]:
A1, coords = make_sensor(N=8, seed=2)
Ai, _ = make_sensor(N=8, seed=3)
Aj, _ = make_sensor(N=8, seed=4)
Ak, _ = make_sensor(N=8, seed=5)

A = QMatrix([A1, Ai, Aj, Ak])

In [ ]:
plot_graph(
    A.abs(), coords=coords,
    figsize=(8, 4), colormap='viridis',
    node_size=120)

In [ ]:
A.visualize()

In [ ]:
eigq, Vq = A.eigendecompose()
new = QuaternionSignal()
new.samples = eigq
eigc = new.to_array()[:, 0] + 1j * new.to_array()[:, 1]

plt.plot(np.real(eigc), np.imag(eigc), 'bo')
plt.show()

### Are we confident that A @ V = V * lambda here?

In [ ]:
(A * Vq[:, 0])[:4]

In [ ]:
(Vq[:, 0] * eigq[0])[:4]

In [ ]:
X_shifted = A * Vq
diff = X_shifted - Vq
diff_norm_squared = (diff.conjugate().transpose() * diff).diag()

In [ ]:
Vq.shape

In [ ]:
diff_norm_squared

In [ ]:
tv = np.sqrt(np.abs(diff_norm_squared).astype(float))

plt.scatter(np.real(eigc), np.imag(eigc), c=tv)
plt.colorbar()
plt.title("Total Variation of eigenvectors for each eigenvalue")
plt.xlabel("Real(eigvals)")
plt.ylabel("Imag(eigvals)")
plt.show()

In [ ]:
idx_freq = np.argsort(tv)

plt.figure()
plt.scatter(np.arange(len(idx_freq)), tv[idx_freq], c=tv[idx_freq])
plt.xlabel("Index of eigenvalues")
plt.ylabel("TV")
plt.title("Sorting eigenvalues by TV")
plt.colorbar()
plt.show()

In [ ]:
h_ideal = np.zeros(len(idx_freq))

# Bandwith of 20% the frequency support
bandwidth = int(len(idx_freq) / 5)
h_ideal[idx_freq[:bandwidth]] = 1

plt.figure()
plt.scatter(np.arange(len(idx_freq)), h_ideal[idx_freq], c=tv[idx_freq])
plt.xlabel("Index of eigenvalues")
plt.ylabel("Frequency response")
cbar = plt.colorbar()
cbar.set_label("TV of respective eigenvector", rotation=90)
plt.title("Ideal LPF")
plt.show()

In [ ]:
from gspx.utils.gsp import gft, igft

# Heat kernel
k = 0.2
ss = np.exp(-k * np.arange(len(idx_freq)))

In [ ]:
ss

Eu quero calcular a inversa de Vq, a matriz de autovetores quaterniônicos, para poder gerar `ss = gft(A, s)`. Como fazer isso?

Talvez: https://www.ime.unicamp.br/sites/default/files/pesquisa/relatorios/rp-1999-45.pdf

In [ ]:
s = Vq.matrix @ ss[:, np.newaxis]

obj = QuaternionSignal()
obj.samples = s.ravel()
node_color = [tuple(rgba) for rgba in obj.to_rgba()]

plot_graph(
    A.abs(), coords=coords, colors=s,
    figsize=(8, 4), colormap='viridis',
    node_size=120)

In [ ]:
rnd = np.random.RandomState(seed=42)
err_amplitude = 0.2
nn = rnd.uniform(low=-err_amplitude, high=err_amplitude, size=len(ss))

sn = igft(A, ss + nn)

plt.figure()
plt.plot(np.arange(len(idx_freq)), ss, 'b.', label="Signal")
plt.plot(np.arange(len(idx_freq)), ss + nn, 'r.', label="Signal + Noise")
plt.xlabel("Index of eigenvalues")
plt.title("Signal and noise spectrum")
plt.legend(loc="upper right")
plt.show()